In [ ]:
import openai
import os
org_key_file = 'org_key'
api_key_file = 'api_key'
openai.organization = open(org_key_file).read().strip()
openai.api_key = open(api_key_file).read().strip()

In [ ]:
from helpers import *
from sklearn.model_selection import train_test_split

path = "data/ner_annotations.json"
sentences, all_labels, label_list, label_dict = load_data(path)

X_train, X_test, y_train, y_test = train_test_split(
    sentences,
    all_labels,
    test_size=0.66666,
    random_state=1
    )

X_test, X_val, y_test, y_val = train_test_split(
    X_test,
    y_test,
    test_size=0.5,
    random_state=1
    )

In [ ]:
responses = []
log = {
    "n_responses": 0,
    "completion_tokens": 0,
    "prompt_tokens": 0,
    "total_tokens": 0,
    "errors" : []
    }

In [ ]:
class_exemplars = {}
for seed_class in label_list:
    class_exemplars[seed_class] = [
        (sentence, labels)
        for sentence, labels in zip(X_train, y_train)
        if seed_class in labels
        ]

In [ ]:
import random

seed_class1 = "nonrecipe-material"
seed_class2 = "amount-misc"
id1 = random.sample(range(len(class_exemplars[seed_class1])), 1)
id2 = random.sample(range(len(class_exemplars[seed_class2])), 1)

labels1 = class_exemplars[seed_class1][id1[0]][1]
exemplar1 = tag_exemplar(class_exemplars[seed_class1][id1[0]][0], labels1)
labels2 = class_exemplars[seed_class2][id2[0]][1]
exemplar2 = tag_exemplar(class_exemplars[seed_class2][id2[0]][0], labels2)

context_prompt = create_context_prompt(
    exemplar1,
    labels1,
    exemplar2,
    labels2,
    )
print(context_prompt)

You are given this set of entities:
condition-misc
precursor
number
unspecified-material
amount-unit
nonrecipe-material
amount-misc
material-descriptor
solvent
operation

The entities are tagged in the examples in an XML-style.

Example 1: Upon <operation> addition </operation> of the <nonrecipe-material> toluene </nonrecipe-material> <material-descriptor> solution </material-descriptor> an <unspecified-material> emulsion </unspecified-material> was <operation> formed </operation> in the vial by <condition-misc> manual </condition-misc> <operation> shaking </operation>.

Example 2: The <amount-misc> stoichiometric amount </amount-misc> of starting <unspecified-material> materials </unspecified-material> , such as <precursor> cobaltous acetate </precursor> and <precursor> ferric chloride </precursor> were <operation> taken </operation> in <number> 1 : 1 </number> , <number> 1 : 2 </number> , <number> 1 : 3 </number> and <number> 1 : 4 </number> <amount-unit> M ratios </amount-unit> , re

In [1]:
# context = openai.ChatCompletion.create(
#     model="gpt-3.5-turbo",
#     messages=[{"role" : "user", "content" : context_prompt}],
#     max_tokens=1000,
#     temperature=0.7,
#     n=1
#     )
# print(context["choices"][0]["message"]["content"])
import torch
with open("./relation_extractions/20230802-120100_entitiy_relations.pt", 'rb') as f:
    responses = torch.load(f)

print(len(responses))
for s in responses:
    print()
    print(s["choices"][0]["message"]["content"])

NameError: name 'extract_augmentations' is not defined

In [ ]:
descriptions = {
    'apparatus-unit': "A unit of measurement used to quantify a capacity or dimension of a synthesis-apparatus. For example, 'kV' can be the unit of measurement for the accelerating voltage of an X-ray measurement apparatus or 'mL' as the unit quantifying the capacity of an autoclave",
    'characterization-apparatus': "A tool or instrument used for characterizing or analyzing the properties or characteristics of a material. For example, an 'X-ray measurement' apparatus used for elemental mapping or 'XRF' indicating that X-ray fluorescence spectroscopy was used to analyze the chemical composition of a precursor",
    'apparatus-property-type': "A specific type of property or characteristic associated with an apparatus. For example, 'capacity' is a property type associated with an autoclave or the 'amplitude' representing the intensity or strength of a sonication.",
    'condition-type': "This entity refers to a specific condition or parameter related to an experiment or process. For example, 'applied voltage' is a condition type associated with the electrospinning process.",
    'number': "Numerical values. For example, words like '14.7', '1', '2.5', and '10000' are number entities as representing specific quantities or measurements.",
    'unspecified-material': "A material that is not explicitly specified or described. For example, 'solution' and 'mixture' refer to unspecified materials.",
    'material': "A substance or matter that can undergo chemical reactions or be used to create products. For example, 'MnP' is a specific material used for membranes.",
    'synthesis-apparatus': "The equipment used for synthesis or production of a material. For example, 'syringe pump' and 'jet' are synthesis apparatuses used in the electrospinning process.",
    'material-descriptor': "Descriptions or characteristics of a substance or material. Words like 'green' and 'poly(vinyl alcohol)' describe the nature of the materials involved. Or 'aqueous solution' can be a material descriptor for an unspecified-material.",
    'condition-unit': "Units of measurement for specific conditions or processes. For example, 'K' (Kelvin) and 'h' (hour) describe the temperature and duration of the sintering process.",
    'operation': "Actions or processes performed on the entities or substances. For example, words like 'dissolved', 'stirring', and 'mixed' are all examples of operations carried out during the synthesis or reaction processes.",
    'apparatus-descriptor': "Describes the characteristics or properties of a specific apparatus or equipment. For example, '200-mesh' specifies the mesh size of a screen used during the sieving operation.",
    'precursor': "A substance or compound that is used as a starting material or ingredient in a chemical reaction or synthesis process. For example, words like 'manganese acetate tetrahydrate' and 'oleic acid' can be precursors if they are the starting materials for a chemical reaction or process.",
    'condition-misc': "Miscellaneous conditions or factors that are relevant to the context. For example, 'magnetic' describes the condition under which the stirring is performed and 'vigorous' and 'homogeneous' are mentioned as miscellaneous conditions describing the stirring and dispersion of precursors.",
    'property-misc': "Miscellaneous properties or characteristics of a substance or material. For example, 'cobalt-based' and 'catalyst' describe the nature of a reference material such as 'Au / AC'.",
    'target': "The desired or intended outcome or product of a process or reaction. For example, 'Au / AC', 'Co / SiO2' and 'zinc oxide' represent target entities if they are the desired material to be prepared. Targets are often mentioned in the title or headline of a recipe.",
    'amount-unit': "The unit of measurement for a quantity. For example, words like 'g', 'ml', 'L', 'μg mL-1', 'wt %', and 'MW' are specifying the unit in which a quantity is measured.",
    'reference': "Citations of a source that provides additional information or support for a statement or procedure. For example, 'Biella et al. 12' or '[12]' represent reference entities linking to a source.",
    'solvent': "A substance in which another substance is dissolved or mixed, typically in liquid form. 'methanol', 'water', 'H2O' or 'HF' are examples of solvents.",
    'gas': "Represents a type of gas used in a chemical process or reaction. For example, 'argon' is the gas used during a heating process.",
    'property-type': "A specific characteristic or quality of a material that can be observed. For example, 'diameter' and 'thickness' are used to describe the dimensions of pellets, while 'morphology' refers to the shape or form of nanostructures.",
    'property-unit': "Represents the unit of measurement used to quantify a property. For example, 'mm' (millimeter) is used to measure the diameter and thickness of pellets.",
    'brand': "The brand or manufacturer of materials or equipment. For example, 'Bunkokeiki' is the brand of a characterization apparatus used for measurements.",
    'meta': "A word or phrase that provides additional information or context about a process or experiment. For example, 'simple hydrothermal method' is meta-information on the preparation method for ferromagnetic nano-sized material.",
    'nonrecipe-material': 'This entity type refers to materials that are not part of a specified recipe or procedure. For example, "toluene" is tagged as nonrecipe-material because it is mentioned as a material added without being part of a specific recipe.',
    'amount-misc': 'This entity type refers to miscellaneous mentions of quantities or amounts. For example, "stoichiometric amount" and "required amount" are tagged as amount-misc because they indicate unspecified quantities of starting materials and solvent, respectively.'
}